Let's move forward with **Few-Shot prompting**. The idea behind few shot prompting is that you can create some examples on how the LLM should proceed when a new question arises.

In [ ]:
from dotenv import load_dotenv, find_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI

load_dotenv(find_dotenv(), override=True)

# Select a model
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

Let's create a template to our Question-Answer agent. 

In [5]:
from langchain_core.prompts import PromptTemplate

question_template = PromptTemplate.from_template("Question: {question}")

Now we can simply invoke it and see the response.

In [6]:
chain = question_template | llm

response = chain.invoke(input={"question" : "Who lived longer, Muhammad Ali or Alan Turing?"})
print(response.content)

Muhammad Ali lived longer than Alan Turing.

*   **Muhammad Ali:** 1942-2016 (74 years)
*   **Alan Turing:** 1912-1954 (41 years)


Not exactly what we where looking for. We would like something more precise, just the name for example. We can try to instruct the model with the system prompt but we could also just simply exemplify the process and that way it might get how we would like to get the answer as well as the process to get there.

In [8]:
question_template = PromptTemplate.from_template("Question: {question}\n{answer}")

examples = [
    {
        "question": "Who lived longer, Muhammad Ali or Alan Turing?",
        "answer": """
Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali
""",
    },
    {
        "question": "When was the founder of craigslist born?",
        "answer": """
Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952
""",
    },
    {
        "question": "Who was the maternal grandfather of George Washington?",
        "answer": """
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washington?
Intermediate answer: The father of Mary Ball Washington was Joseph Ball.
So the final answer is: Joseph Ball
""",
    },
    {
        "question": "Are both the directors of Jaws and Casino Royale from the same country?",
        "answer": """
Are follow up questions needed here: Yes.
Follow up: Who is the director of Jaws?
Intermediate Answer: The director of Jaws is Steven Spielberg.
Follow up: Where is Steven Spielberg from?
Intermediate Answer: The United States.
Follow up: Who is the director of Casino Royale?
Intermediate Answer: The director of Casino Royale is Martin Campbell.
Follow up: Where is Martin Campbell from?
Intermediate Answer: New Zealand.
So the final answer is: No
""",
    },
]

[FewShotPromptingTemplate](https://python.langchain.com/docs/how_to/few_shot_examples/) allows to structure the query so that we do not have to repeat the process to make it simpler to code.

In [15]:
from langchain_core.prompts import FewShotPromptTemplate

prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=question_template,
    suffix="Question: {input}",
    input_variables=["input"],
)

chain = prompt | llm

response = chain.invoke({"input": "Would you rather use LangChain or LlamaIndex?"})
print(response.content)

Are follow up questions needed here: Yes.
Follow up: What are the strengths of LangChain?
Intermediate Answer: LangChain is strong in its versatility, supporting a wide range of applications and integrations with various tools and models.
Follow up: What are the strengths of LlamaIndex?
Intermediate Answer: LlamaIndex excels in data indexing and retrieval, particularly for knowledge graphs and unstructured data sources, making it suitable for question answering and knowledge discovery tasks.
Follow up: What kind of project am I working on?
Intermediate Answer: As an AI, I do not know what kind of project you are working on.
Follow up: What information should I consider to choose between LangChain and LlamaIndex?
Intermediate Answer: Consider the complexity of your project, the types of data you're working with, and the specific functionalities you require. If you need broad versatility and integration capabilities, LangChain might be a better fit. If you need specialized data indexing 

One caveat is that we might what to tag the final answer so that it is easier to extract if that is what we are going to present to the final user.